In [44]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

import pandas as pd
import seaborn as sns

In [45]:
df = pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [46]:
df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

In [47]:
df = pd.get_dummies(df,columns=['Geography','Gender'],drop_first=True)

In [48]:
X = df.drop('Exited', axis=1)
y = df['Exited']

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [50]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [51]:
class ChurnData(Dataset):
    def __init__(self, X_train, y_train) -> None:
        super().__init__()
        self.X = torch.from_numpy(X_train)
        self.y = torch.from_numpy(y_train).view(-1,1)
        # self.y = self.y.type(torch.LongTensor)
        # self.len = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return self.len


In [52]:
churn_data = ChurnData(X_train.values, y_train.values)
train_loader = DataLoader(dataset=churn_data, batch_size=32)

In [53]:
print(f"X Shape: {churn_data.X.shape}, y shape: {churn_data.y.shape}")

X Shape: torch.Size([8000, 11]), y shape: torch.Size([8000, 1])


In [54]:
class ChurnModel(nn.Module):
    def __init__(self, input_features, output_features,hidden_features) -> None:
        super().__init__()
        self.layer1 = nn.Linear(input_features, hidden_features)
        self.layer2 = nn.Linear(hidden_features, hidden_features)
        # self.layer3 = nn.Linear(hidden_features, output_features)
        self.sigmoid = nn.Sigmoid()
        # self.relu = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        # x = self.relu(x)
        x = self.layer2(x)
        # x = self.relu(x)
        # x = self.layer3(x)
        x = self.sigmoid(x)
        return x

In [55]:
input_features = X_train.shape[1]
output_features = 1
hidden_features = 10
model = ChurnModel(input_features, output_features, hidden_features)

In [56]:
from torchsummary import summary

summary(model, input_size=(input_features,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                   [-1, 10]             120
            Linear-2                   [-1, 10]             110
           Sigmoid-3                   [-1, 10]               0
Total params: 230
Trainable params: 230
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [57]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [58]:
NUM_EPOCHS = 100
losses = []
for epoch in range(NUM_EPOCHS):
    for x, y in train_loader:

        # initialize gradients
        optimizer.zero_grad()

        # forward pass
        y_hat_log = model(x)

        # reshape y to match y_hat_log
        y = y.view(-1, 1).float()

        # calculate losses
        loss = criterion(y_hat_log, y)

        # calculate gradients
        loss.backward()

        # update parameters
        optimizer.step()

    losses.append(float(loss.data.detach().numpy()))

AttributeError: 'ChurnData' object has no attribute 'len'